In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def semor2(a, b):
    dst = np.zeros((len(a), len(b)))
    ind = np.zeros((len(a), len(b)))
    
    for i in range(len(a) - 1, -1, -1):
        for j in range(len(b) - 1, -1, -1):
            cur = 1e9
            
            if (i != len(a) - 1):
                cur = dst[i + 1][j]
                ind[i][j] = 0
                
            if (j != len(b) - 1 and cur > dst[i][j + 1]):
                cur = dst[i][j + 1]
                ind[i][j] = 1
                
            if (i == len(a) - 1 and j == len(b) - 1):
                cur = 0
                
            if (i != len(a) - 1 and j != len(b) - 1 and cur > dst[i + 1][j + 1]):
                cur = dst[i + 1][j + 1]
                ind[i][j] = 2
                
            cur += np.power(a[i] - b[j], 2)
            
            dst[i][j] = cur
    
    pos = []
    x = 0
    y = 0
    
    while (x != len(a) - 1 or y != len(b) - 1):
        pos.append((x, y))
        
        if (ind[x][y] == 0):
            x += 1
        elif (ind[x][y] == 1):
            y += 1
        else:
            x += 1
            y += 1
            
    pos.append((x, y))
    
    return (np.array(pos), dst[0][0])

def pol_dist2(a, b):
    a = a * 1.
    a -= np.mean(a)
    a = a / np.std(a)
    b = b * 1.
    b -= np.mean(b)
    b = b / np.std(b)
    pos, d = semor2(a, b)
    x = a[pos[:,0]]
    y = b[pos[:,1]]
    k, e = np.polyfit(y, x, 1)
    
    return np.sum(np.power(x - k * y - e, 2))

In [3]:
def semor3(a, b, c):
    dst = np.zeros((len(a), len(b), len(c)))
    ind = np.zeros((len(a), len(b), len(c)))
    
    for i in range(len(a) - 1, -1, -1):
        for j in range(len(b) - 1, -1, -1):
            for k in range(len(c) - 1, -1, -1):
                cur = 1e9

                if (i != len(a) - 1):
                    cur = dst[i + 1][j][k]
                    ind[i][j][k] = 0

                if (j != len(b) - 1 and cur > dst[i][j + 1][k]):
                    cur = dst[i][j + 1][k]
                    ind[i][j][k] = 1

                if (i == len(a) - 1 and j == len(b) - 1 and k == len(c) - 1):
                    cur = 0

                if (i != len(a) - 1 and j != len(b) - 1 and cur > dst[i + 1][j + 1][k]):
                    cur = dst[i + 1][j + 1][k]
                    ind[i][j][k] = 2
                    
                if (k != len(c) - 1 and cur > dst[i][j][k + 1]):
                    cur = dst[i][j][k + 1]
                    ind[i][j][k] = 3
                    
                if (i != len(a) - 1 and k != len(c) - 1 and cur > dst[i + 1][j][k + 1]):
                    cur = dst[i + 1][j][k + 1]
                    ind[i][j][k] = 4
                    
                if (j != len(b) - 1 and k != len(c) - 1 and cur > dst[i][j + 1][k + 1]):
                    cur = dst[i][j + 1][k + 1]
                    ind[i][j][k] = 5
                
                if (i != len(a) - 1 and j != len(b) - 1 and k != len(c) - 1 and cur > dst[i + 1][j + 1][k + 1]):
                    cur = dst[i + 1][j + 1][k + 1]
                    ind[i][j][k] = 6

                cur += np.power(a[i] - b[j] - c[k], 2)

                dst[i][j][k] = cur
    
    pos = []
    x = 0
    y = 0
    z = 0
    
    while (x != len(a) - 1 or y != len(b) - 1 or z != len(c) - 1):
        pos.append((x, y, z))
        
        if (ind[x][y][z] == 0):
            x += 1
        elif (ind[x][y][z] == 1):
            y += 1
        elif (ind[x][y][z] == 2):
            x += 1
            y += 1
        elif (ind[x][y][z] == 3):
            z += 1
        elif (ind[x][y][z] == 4):
            x += 1
            z += 1
        elif (ind[x][y][z] == 5):
            y += 1
            z += 1
        else:
            x += 1
            y += 1
            z += 1
            
    pos.append((x, y, z))
    
    return (np.array(pos), dst[0][0][0])

def pol_dist3(a, b, c):
    pos, d = semor3(a, b, c)
    #return d
    x = a[pos[:,0]]
    y = b[pos[:,1]]
    z = c[pos[:,2]]
    k, e = np.polyfit(y + z, x, 1)
    
    return np.mean(np.power(x - k * (y + z) - e, 2))

In [4]:
from tqdm import tqdm
numb = [0] * 16

for p in range(10):
    grid = np.random.rand(10) * 10 + 10
    a = [grid / 8, np.sin(grid), np.log(grid), np.exp(grid / 10)]

    b = [np.cos(grid), np.sin(np.log(grid)), np.sin(grid * 2), np.tan(grid / 5) * 2]
    c = []

    for i in range(len(a)):
        for j in range(len(b)):
            c.append(a[i] + b[j])

            c[-1] = np.concatenate((c[-1], c[-1], c[-1], c[-1], c[-1]))


    for i in range(len(a)):
        a[i] = np.concatenate((a[i], a[i], a[i], a[i]))
        b[i] = np.concatenate((b[i], b[i], b[i], b[i]))

    for k in tqdm(range(len(c))):
        i1 = 0
        i2 = 0
        best = 1e9

        for i in range(len(a)):
            for j in range(len(b)):
                cur = pol_dist3(c[k], a[i], b[j])
                if (cur < best):
                    best = cur
                    i1 = i
                    i2 = j

        if (i1 * 4 + i2 == k):
            numb[k] += 1

100%|██████████| 16/16 [02:27<00:00,  9.23s/it]


In [17]:
print('total acc', np.sum(numb) / 160)
for i in range(16):
    print("acc for ", i // 4, i % 4, " = ", numb[i] / 10)

total acc 0.825
acc for  0 0  =  1.0
acc for  0 1  =  1.0
acc for  0 2  =  0.9
acc for  0 3  =  0.8
acc for  1 0  =  1.0
acc for  1 1  =  0.8
acc for  1 2  =  0.9
acc for  1 3  =  1.0
acc for  2 0  =  0.9
acc for  2 1  =  1.0
acc for  2 2  =  0.8
acc for  2 3  =  0.5
acc for  3 0  =  0.6
acc for  3 1  =  0.4
acc for  3 2  =  0.6
acc for  3 3  =  1.0
